In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import torch

import sys
sys.path.append('../prepare_dataset')
sys.path.append('../models')
sys.path.append('../post_training')
import data_aug
import prepare_binary_dataset
import data_aug
import models_classification
import utils
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns

### Derive features from the input image: for reproductiblity

In [ ]:
'''
img_df=pd.read_csv('../metadata/metadata.csv')
DATA_DIR='../../../data/eardrum_public_data'
relative_path=img_df['relative_file_path'].values
subfolder_name=img_df['source'].values
img_paths=DATA_DIR+'/'+subfolder_name+relative_path
df_feature=pd.DataFrame(columns=['hue_mean','hue_std','saturation_mean','saturation_std','value_mean','value_std'])
for ii,save_path in enumerate(img_paths):
    image=cv2.imread(save_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    features=[hsv_image[:,:,0].mean(),hsv_image[:,:,0].std(),
    hsv_image[:,:,1].mean(),hsv_image[:,:,1].std(),
    hsv_image[:,:,2].mean(),hsv_image[:,:,2].std()]
    df_feature.loc[ii] =features
img_df2=pd.concat([img_df,df_feature],axis=1)
img_df2.to_csv('../metadata/metadata_wfeatures.csv',index=False)
'''

### Load features

In [ ]:
img_df=pd.read_csv('../metadata/metadata_wfeatures.csv')
seed=1234

### Logistic regression using the HSV feature set

In [ ]:
df_metrics=pd.DataFrame(columns=['train_origin','feature_type',
                                 'val_acc', 'val_auc',
                                 'test0_acc','test0_auc',
                                 'test1_acc','test1_auc'])
selected_features=['hue_mean','hue_std','saturation_mean','saturation_std','value_mean','value_std']
for index in range(3): 
    origins=['Chile','Ohio','Turkey']
    train_origin=origins[index]
    print(train_origin)
    train0=img_df[img_df.source==origins.pop(index)]
    test0=img_df[img_df.source==origins[0]]
    test1=img_df[img_df.source==origins[1]]
    print('test0 test1:',origins[0],origins[1])
    if train_origin=='Ohio':
        train,val=train_test_split(train0,stratify=train0['class'],test_size=0.2,shuffle=True,random_state=seed)
    else:
        train=train0[train0.is_test==False].reset_index(drop=True)
        val=train0[train0.is_test==True].reset_index(drop=True)
    # train logistic regression model using 'hue_mean', 'hue_std', 'saturation_mean', 'saturation_std','value_mean', 'value_std'
    X_train=train[selected_features]
    y_train=train['binary_class']
    X_val=val[selected_features]
    y_val=val['binary_class']
    X_test0=test0[selected_features]
    y_test0=test0['binary_class']
    X_test1=test1[selected_features]
    y_test1=test1['binary_class']
    clf = LogisticRegression().fit(X_train, y_train)
    print('coef',clf.coef_)
    y_pred=clf.predict(X_val)
    acc_val=metrics.accuracy_score(y_val, y_pred)
    print('val accuracy:',acc_val)
    auc_val=metrics.roc_auc_score(y_val, clf.predict_proba(X_val)[:,1])
    print('val auc:',auc_val)
    y_pred=clf.predict(X_test0)
    acc_test0=metrics.accuracy_score(y_test0, y_pred)
    auc_test0=metrics.roc_auc_score(y_test0, clf.predict_proba(X_test0)[:,1])
    print('test0 accuracy:',acc_test0)
    print('test0 auc:',auc_test0)
    y_pred=clf.predict(X_test1)
    acc_test1=metrics.accuracy_score(y_test1, y_pred)
    auc_test1=metrics.roc_auc_score(y_test1, clf.predict_proba(X_test1)[:,1])
    print('test1 accuracy:',acc_test1)
    print('test1 auc:',auc_test1)
    df_metrics.loc[index]=[train_origin,'all',acc_val,auc_val,
                                      acc_test0,auc_test0,
                                      acc_test1,auc_test1]
    print('')


### Logistic regression using the single feature set

In [ ]:
df_metrics2=pd.DataFrame(columns=['train_origin','feature_type',
                                 'val_acc', 'val_auc',
                                 'test0_acc','test0_auc',
                                 'test1_acc','test1_auc'])
for index in range(3): 
    origins=['Chile','Ohio','Turkey']
    train_origin=origins[index]
    print(train_origin)
    train0=img_df[img_df.source==origins.pop(index)]
    test0=img_df[img_df.source==origins[0]]
    test1=img_df[img_df.source==origins[1]]
    print('test0 test1:',origins[0],origins[1])
    if train_origin=='Ohio':
        train,val=train_test_split(train0,stratify=train0['class'],test_size=0.2,shuffle=True,random_state=seed)
    else:
        train=train0[train0.is_test==False].reset_index(drop=True)
        val=train0[train0.is_test==True].reset_index(drop=True)
    # train logistic regression model using 'hue_mean', 'hue_std', 'saturation_mean', 'saturation_std','value_mean', 'value_std'
    X_train=train[['saturation_std']]
    y_train=train['binary_class']
    X_val=val[['saturation_std']]
    y_val=val['binary_class']
    X_test0=test0[['saturation_std']]
    y_test0=test0['binary_class']
    X_test1=test1[['saturation_std']]
    y_test1=test1['binary_class']
    clf = LogisticRegression().fit(X_train, y_train)
    y_pred=clf.predict(X_val)
    print('coef',clf.coef_)
    # get confidence interval of the coefficients

    
    acc_val=metrics.accuracy_score(y_val, y_pred)
    print('val accuracy:',acc_val)
    auc_val=metrics.roc_auc_score(y_val, clf.predict_proba(X_val)[:,1])
    print('val auc:',auc_val)
    y_pred=clf.predict(X_test0)
    acc_test0=metrics.accuracy_score(y_test0, y_pred)
    auc_test0=metrics.roc_auc_score(y_test0, clf.predict_proba(X_test0)[:,1])
    print('test0 accuracy:',acc_test0)
    print('test0 auc:',auc_test0)
    y_pred=clf.predict(X_test1)
    acc_test1=metrics.accuracy_score(y_test1, y_pred)
    auc_test1=metrics.roc_auc_score(y_test1, clf.predict_proba(X_test1)[:,1])
    print('test1 accuracy:',acc_test1)
    print('test1 auc:',auc_test1)

    df_metrics2.loc[index]=[train_origin,'saturation_std',acc_val,auc_val,
                                      acc_test0,auc_test0,
                                      acc_test1,auc_test1]
    print('')


## Visualization

In [ ]:
df_metrics=pd.concat([df_metrics,df_metrics2],axis=0)
df_metrics_long=pd.melt(df_metrics, id_vars=['train_origin','feature_type'], 
value_vars=['val_acc','val_auc','test0_acc','test0_auc','test1_acc','test1_auc'
], var_name='metrics', value_name='value')
df_metrics_long_auc=df_metrics_long.loc[df_metrics_long['metrics'].str.contains('_auc')]

In [ ]:
sns.set(font_scale=1.5)

g = sns.catplot(
    data=df_metrics_long_auc[df_metrics_long_auc.train_origin=='Chile'], kind="bar",
    x="metrics", y="value", hue='feature_type',
    palette="Paired", 
     alpha=.6, height=5
)
g.legend.set_title("Input")
new_labels = ['HSV feature set', 'saturation std']
for t, l in zip(g._legend.texts, new_labels): t.set_text(l)

g.set_axis_labels("", "AUC")
g.fig.suptitle('Trained on Chile',horizontalalignment='center', verticalalignment='top')

g.set_xticklabels(["Internal", "Ohio", "Turkey"])
g.set(ylim=(0, 1))
g._legend.remove()

In [ ]:
sns.set(font_scale=1.5)

g = sns.catplot(
    data=df_metrics_long_auc[df_metrics_long_auc.train_origin=='Ohio'], kind="bar",
    x="metrics", y="value", hue='feature_type',
    palette="Paired", 
     alpha=.6, height=5
)
g.legend.set_title("Input")
new_labels = ['HSV feature set', 'saturation std']
for t, l in zip(g._legend.texts, new_labels): t.set_text(l)

g.set_axis_labels("", "AUC")
g.fig.suptitle('Trained on Ohio')

g.set_xticklabels(["Internal", "Chile", "Turkey"])
g.set(ylim=(0, 1))
g._legend.remove()


In [ ]:
sns.set(font_scale=1.5)

g = sns.catplot(
    data=df_metrics_long_auc[df_metrics_long_auc.train_origin=='Turkey'], kind="bar",
    x="metrics", y="value", hue='feature_type',
    palette="Paired", 
     alpha=.6, height=5
)
g.legend.set_title("Input")
new_labels = ['HSV features', 'single feature']
for t, l in zip(g._legend.texts, new_labels): t.set_text(l)

g.set_axis_labels("", "AUC")
g.fig.suptitle('Trained on Turkey')

g.set_xticklabels(["Internal", "Chile", "Ohio"])
g.set(ylim=(0, 1))